# Packages et chemins

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
repertoire_data = 'Data\\'
fichier_meteo = 'weather_out.csv'

# Fichier RTE

In [3]:
# Chargement du df principal, toutes les années, données agrégées au pas journalier
 
df_rte = pd.read_csv('Data\rte_jour_regions.csv', sep=';', encoding='utf-8')

# Chargement du df avec toutes les années, données agrégées au pas mensuel
df_rte_mois = pd.read_csv('Data\rte_mois_regions.csv', sep=';', encoding='utf-8')

OSError: [Errno 22] Invalid argument: 'Data\rte_jour_regions.csv'

In [ ]:
print(df_rte.info())
df_rte.head()

## Cartes

In [ ]:
import geopandas as gpd
# import geoplot
# import geoplot.crs as gcrs

In [ ]:
geo_data = gpd.read_file(repertoire_data + 'regions.geojson')

In [ ]:
geo_data.head()

In [ ]:
geo_data = geo_data.drop([9, 10, 11, 12, 13], axis=0)

In [ ]:
# Croisement des data geojson avec la production
df_rte_sum = df_rte[df_rte['Annee']==2019].groupby('Code INSEE région', as_index=False).agg(sum)

In [ ]:
df_rte_sum['Code INSEE région'] = df_rte_sum['Code INSEE région'].astype(int)
geo_data['code'] = geo_data['code'].astype(int)
df_rte_sum = df_rte_sum.merge(geo_data, left_on='Code INSEE région', right_on='code', how='inner')

In [ ]:
geo = gpd.GeoDataFrame(df_rte_sum, geometry='geometry')

In [ ]:
fig, axes = plt.subplots(figsize=(20, 10), ncols=2, nrows=1)
axes[0] = fig.add_subplot(121)
geo.plot(column='Consommation (MW)', cmap='Blues', ax=axes[0])
axes[0].axis('off')
vmin = geo['Consommation (MW)'].min()
vmax = geo['Consommation (MW)'].max()
sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
cbar = fig.colorbar(sm)
axes[0].set_title('Consommation par région')

axes[1] = fig.add_subplot(122)
geo.plot(column='Production totale (MW)', cmap='Blues', ax=axes[1])
axes[1].axis('off')
vmin = geo['Production totale (MW)'].min()
vmax = geo['Production totale (MW)'].max()
sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
cbar = fig.colorbar(sm)
axes[1].set_title('Production par région');

Les cartes ci-dessus montrent la consommation et la production pour chaque région. On constate que dans certaines régions, la production est faible voire inexistante, mais la consommation est forte (par exemple l'Ile de France).

## Catplot consommation - production

In [ ]:
sns.catplot(x='Code INSEE région', y='Production totale (MW)', kind='box', col='Annee', data=df_rte)
sns.catplot(x='Code INSEE région', y='Consommation (MW)', kind='box', col='Annee', data=df_rte);

La conclusion est la même que l'analyse avec les cartes. On note en plus une constance d'une année sur l'autre.

## Exploration des données

### Calcul de la consommation totale par an

In [ ]:
for annee in range(2013, 2022):
    conso_annuelle = df_rte[df_rte['Annee']==annee]['Consommation (MW)'].sum()/2
    print('Conso totale', annee, ':', int(conso_annuelle/1000000), 'TWh')

## Visualisation de données

### Productions mensuelles empilées par filière, pour une année choisie

In [ ]:
annee = 2019

plt.figure(figsize=(12, 10))

plt.stackplot(
    'Date',
    'Thermique (MW)',
    'Nucléaire (MW)',
    'Eolien (MW)',
    'Solaire (MW)',
    'Bioénergies (MW)',
    'Hydraulique (MW)',
    data=df_rte[df_rte['Annee']==annee].groupby('Date', as_index=False).agg(sum),
    labels=['Thermique', 'Nucléaire', 'Eolien', 'Solaire', 'Bioénergies', 'Hydraulique'],
    colors=['brown', 'orange', 'green', 'yellow', 'grey', 'blue']
)
plt.ylabel('Puissance (MW)')
plt.xticks([str(annee)+'-01-01', str(annee)+'-04-01', str(annee)+'-07-01', str(annee)+'-10-01'])
plt.title('Production par filière en ' + str(annee))
plt.legend(loc='upper center');

La majorité de la production provient de la filière nucléaire. La consommation d'électricité est nettement plus faible en été qu'en hiver.

### Production mensuelle par filière pour une année choisie

In [ ]:
annee = 2019

fig, axes = plt.subplots(3, 2, figsize=(20, 26))

filieres = ['Thermique (MW)'
            , 'Nucléaire (MW)'
            , 'Eolien (MW)'
            , 'Solaire (MW)'
            , 'Hydraulique (MW)'
            , 'Bioénergies (MW)'
           ]

for (index, filiere), ax in zip(enumerate(filieres), axes.flatten()):
    ax.bar(x=df_rte_mois[df_rte_mois['Annee']==annee]['Mois']
                  , height=df_rte_mois[df_rte_mois['Annee']==annee]['Thermique (MW)'])
    ax.set_ylabel(filiere)
    ax.set_title('Production '+ filiere+ ' en '+str(annee))
    ax.set_xticks([str(annee)+'-01', str(annee)+'-04', str(annee)+'-07', str(annee)+'-10'])

La saisonnalité n'est pas la même selon la filière de production. La production solaire est plus importante l'été, l'éolien connait des pics en hiver.
Le nucléaire connait également une saisonnalité en relation avec la consommation (plus de production l'hiver).

### Différence entre consommation et production et mise en rapport avec les échanges physiques
Application sur l'année 2015 de façon aléatoire.

In [ ]:
annee = 2015

df_rte[df_rte['Annee']==annee].drop(['Annee', 'Jour_semaine', 'Code INSEE région'], axis=1).groupby(
    'Date', as_index=False
).agg(sum).iloc[:, :14]

In [ ]:
df_rte['Conso-Prod'] = df_rte['Consommation (MW)'] - df_rte['Production totale (MW)']

In [ ]:
annee = 2015

fig = plt.figure(figsize=(12,10))

plt.plot(df_rte[df_rte['Annee']==annee].groupby('Date').agg(sum)['Consommation (MW)'], 
         label='Consommation')
plt.plot(df_rte[df_rte['Annee']==annee].groupby('Date').agg(sum)['Production totale (MW)'], 
         linestyle='-', 
         label='Production', 
         color='b'
        )

plt.plot(df_rte[df_rte['Annee']==annee].groupby('Date', as_index=False).agg(sum)['Ech. physiques (MW)'], 
         linestyle='--', 
         label='Ech. physiques', 
         color='grey'
        )
plt.ylabel('MW')
plt.xticks(['2015-01-01', '2015-04-01', '2015-07-01', '2015-10-01'])
plt.title('Consommation, production et échanges physiques')
plt.legend();

--> La différence entre production et consommation pourrait correspondre aux échanges physiques. Pour s'en convaincre on représente le delta entre consommation et production face aux échanges physiques.

In [ ]:
plt.figure(figsize=(12,10))

plt.plot(df_rte[df_rte['Annee']==annee].groupby('Date').agg(sum)['Conso-Prod'], 
         linestyle='-', 
         label='Conso-Prod')
plt.plot(df_rte[df_rte['Annee']==annee].groupby('Date').agg(sum)['Ech. physiques (MW)'], 
         linestyle='-', 
         color='grey', 
         label='Ech. physiques'
        )
plt.ylabel('MW')
plt.xticks(['2015-01-01', '2015-04-01', '2015-07-01', '2015-10-01'])
plt.title('Delta entre consommation et production vs. échanges physiques')
plt.legend();

**--> Les deux courbes sont parfaitement superposées, donc les échanges physiques correspondent bien à la différence entre consommation et production, et reflètent donc les imports et exports aux frontières françaises.**

## Consommation en fonction du jour de la semaine

In [ ]:
df_agg_j = df_rte.groupby(['Annee','Jour_semaine'], as_index=False ).agg({'Consommation (MW)' : 'sum'})          

plt.figure (figsize = (20,10))
plt.plot (df_agg_j['Jour_semaine'][(df_agg_j['Annee']== 2015)],df_agg_j['Consommation (MW)'][(df_agg_j['Annee'] == 2015)], "-gs" ,label = '2015')
plt.plot (df_agg_j['Jour_semaine'][(df_agg_j['Annee'] == 2016)],df_agg_j['Consommation (MW)'][(df_agg_j['Annee'] == 2016)], "-bs",label = '2016')
plt.plot (df_agg_j['Jour_semaine'][(df_agg_j['Annee'] == 2017)],df_agg_j['Consommation (MW)'][(df_agg_j['Annee'] == 2017)], "-rs",label = '2017')
plt.plot (df_agg_j['Jour_semaine'][(df_agg_j['Annee'] == 2018)],df_agg_j['Consommation (MW)'][(df_agg_j['Annee'] == 2018)], "-ys",label = '2018')
plt.plot (df_agg_j['Jour_semaine'][(df_agg_j['Annee'] == 2019)],df_agg_j['Consommation (MW)'][(df_agg_j['Annee'] == 2019)], "-ks",label = '2019')
plt.xticks(range(7), ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche'], rotation=70)
plt.ylabel('Consommation (MW)')
plt.legend();

La consommation baisse fortement le samedi, et encore plus le dimanche, du fait de l'arrêt des gros consommateurs (gros industriels).

# Fichier météo

## Lecture du fichier

In [ ]:
df_meteo = pd.read_csv((repertoire_data + fichier_meteo), sep = ',')

In [ ]:
df_meteo.head()

# Croisement des df RTE et Météo

In [ ]:
df_rte = df_rte.iloc[:, :19]
df_rte.head()

In [ ]:
df_rte_meteo = df_rte.merge(
    right=df_meteo, 
    left_on=['Date', 'Code INSEE région'], 
    right_on=['aaaammjj', 'region'],
    how='inner')
df_rte_meteo = df_rte_meteo.drop(['aaaammjj', 'region', 'joursem', 'annee'], axis=1)
df_rte_meteo.head()

## Corrélations entre variables

In [ ]:
corr = df_rte_meteo.corr()
lower_triangle = np.tril(corr, k = -1)
mask = lower_triangle == 0

plt.figure(figsize = (15,8))  
sns.set_style(style = 'white')   
sns.heatmap(lower_triangle, center=0.5, cmap= 'Blues', annot= True, xticklabels = corr.index, yticklabels = corr.columns,
            cbar= False, linewidths= 1, mask = mask)   # 
plt.xticks(rotation = 50)    
plt.yticks(rotation = 20);

## Relation production éolienne et vitesse du vent

In [ ]:
plt.figure(figsize=(12, 10))
plt.scatter(x=df_rte_meteo[df_rte_meteo['Code INSEE région']==32]['rafale_periode'], 
            y=df_rte_meteo[df_rte_meteo['Code INSEE région']==32]['Eolien (MW)'])
plt.xlabel('Vitesse du vent')
plt.ylabel('Production éolienne (MW)')
plt.title('Production éolienne en fonction du vent');

On remarque clairement la corrélation entre production éolienne et vitesse du vent.

## Relation production solaire et nébulosité

In [ ]:
plt.figure(figsize=(12, 10))
plt.scatter(x=df_rte_meteo[df_rte_meteo['Code INSEE région']==11]['nebulosite'], 
            y=df_rte_meteo[df_rte_meteo['Code INSEE région']==11]['Solaire (MW)'])
plt.xlabel('Nébulosité totale')
plt.ylabel('Production solaire (MW)')
plt.title("Production solaire en fonction de la 'nébulosité totale'");

On voit une corrélation entre la production solaire et la nébulosité, mais le lien n'est pas très net. Une des pistes est de revoir le traitement des valeurs manquantes dans le fichier météo (notamment en affectant une valeur 1 aux tranches 22h-1h et 1h-4h).

## Relation consommation et température

In [ ]:
plt.figure(figsize=(12, 10))
plt.scatter(x=df_rte_meteo[df_rte_meteo['Code INSEE région']==11]['temp'], 
            y=df_rte_meteo[df_rte_meteo['Code INSEE région']==11]['Consommation (MW)'])
plt.xlabel('Température (°C)')
plt.ylabel('Consommation (MW)')
plt.title('Consommation en fonction de la température (°C)');

On voit nettement la hausse de la consommation quand la température diminue. Au-delà d'une température d'environ 20°, la consommation augmente à nouveau du fait de la hausse de la climatisation.